In [1]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

video_features = diffusion_wrapper.extract_video_features('../rocket256.gif', "A rocket starting on Mars.")

/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  8.84it/s]
/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/max/miniconda3/envs/diffusers/lib/python3.11/site-packages/torch/nn/modules/conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered in

torch.Size([25, 3520, 32, 32])


In [ ]:
from diffusers import AutoencoderKL

# Transform feature maps with decoder
vae = AutoencoderKL.from_pretrained("ali-vilab/text-to-video-ms-1.7b", subfolder="vae", use_safetensor=True).to("cuda")
F, C, H, W = video_features.shape
feature_maps_channel_batches = video_features.view(F, C // 4, 4, H, W)
feature_maps_channel_batches_decoded = torch.zeros_like(feature_maps_channel_batches)
for c in range(C // 4):
    feat = feature_maps_channel_batches[:, c, :, :, :].float()
    f_decoded = vae.decode(feat)
    feature_maps_channel_batches_decoded[:, c, :, :, :] = f_decoded
feature_maps_decoded = feature_maps_channel_batches_decoded.view(F, C, H, W)

In [2]:
video_features = video_features.permute(0, 2, 3, 1).float().cpu()
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))

#heatmaps = torch.permute(heatmaps, (0, 3, 1, 2))
#heatmaps = torch.nn.functional.interpolate(heatmaps, size=256, mode="bilinear", align_corners=True) * 255
#
#heatmaps = heatmaps.squeeze().numpy()
#frames_gif = [Image.fromarray(f) for f in heatmaps]
#frames_gif[0].save("output/heatmaps.gif", save_all=True, append_images=frames_gif[1:], duration=100, loop=0)

tracks = zero_shot_tracker.track(heatmaps)

def load_frames(image: Image, mode='RGB'):
    return np.array([
        np.array(frame.convert(mode))
        for frame in ImageSequence.Iterator(image)
    ])

with Image.open('../rocket256.gif') as im:
    frames = load_frames(im)

zero_shot_tracker.place_marker_in_frames(frames, tracks)

projected point: 
(11.625, 17.125, 0)
sample feat vec
tensor([[[[-0.3051]],

         [[-3.4736]],

         [[-0.8722]],

         ...,

         [[ 2.2353]],

         [[ 0.9984]],

         [[-5.2600]]]])
